In [2]:

import numpy as np
import os


In [ ]:
text_arousal_path = 'D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/Saves/Text_CL_arousal_lst.npy'
music_arousal_path = 'D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/Saves/Music_CL_arousal_lst.npy'

text_valence_path = 'D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/Saves/Text_CL_valence_lst.npy'
music_valence_path = 'D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/Saves/Music_CL_valence_lst.npy'



In [3]:
music_arousal = np.load(music_arousal_path)
music_valence = np.load(music_valence_path)

text_arousal = np.load(text_arousal_path)
text_valence = np.load(text_valence_path)

music_dataset = [
    {
        "index": i,
        "arousal": float(arousal),
        "valence": float(valence) 
    }
    for i, (arousal, valence) in enumerate(zip(music_arousal, music_valence))
]

text_dataset = [
    {
        "index": i,
        "arousal": float(arousal),
        "valence": float(valence) 
    }
    for i, (arousal, valence) in enumerate(zip(text_arousal, text_valence))
]

print("Music Dataset:", music_dataset[:5]) 
print("Text Dataset:", text_dataset[:5])    

Music Dataset: [{'index': 0, 'arousal': 4.333333333, 'valence': 5.833333333}, {'index': 1, 'arousal': 6.0, 'valence': 8.0}, {'index': 2, 'arousal': 8.333333333, 'valence': 6.333333333}, {'index': 3, 'arousal': 3.833333333, 'valence': 3.0}, {'index': 4, 'arousal': 3.333333333, 'valence': 6.5}]
Text Dataset: [{'index': 0, 'arousal': 5.5, 'valence': 4.92}, {'index': 1, 'arousal': 4.18, 'valence': 4.65}, {'index': 2, 'arousal': 7.42, 'valence': 5.77}, {'index': 3, 'arousal': 4.54, 'valence': 3.28}, {'index': 4, 'arousal': 3.42, 'valence': 5.59}]


In [4]:
print("Music Dataset Length:", len(music_dataset))
print("Text Dataset Length:", len(text_dataset))

Music Dataset Length: 985
Text Dataset Length: 6832


In [6]:
import random

save_path = 'D:/PolyU/URIS/Part2_projects/WEMOM_V1/Text2Music/Samples/'

n = 4  
num_anchors = 100 
train_ratio = 0.8 

def calculate_similarity(valence1, arousal1, valence2, arousal2):
    return ((valence1 - valence2) ** 2 + (arousal1 - arousal2) ** 2) ** 0.5

def save_samples_to_folder(samples, filename):
    filepath = os.path.join(save_path, filename)
    with open(filepath, "w") as f:
        for sample in samples:
            f.write(" ".join(map(str, sample)) + "\n")

def build_samples(text_dataset, music_dataset, n, num_anchors):
    anchors = random.sample(text_dataset, num_anchors)  
    text_pos_samples = []
    text_neg_samples = []
    music_pos_samples = []
    music_neg_samples = []

    for anchor in anchors:
        anchor_index = anchor["index"]
        anchor_valence = anchor["valence"]
        anchor_arousal = anchor["arousal"]

        music_similarities = [
            {
                "index": music["index"],
                "similarity": calculate_similarity(
                    anchor_valence, anchor_arousal, music["valence"], music["arousal"]
                ),
            }
            for music in music_dataset
        ]
        music_similarities.sort(key=lambda x: x["similarity"]) 

        text_similarities = [
            {
                "index": text["index"],
                "similarity": calculate_similarity(
                    anchor_valence, anchor_arousal, text["valence"], text["arousal"]
                ),
            }
            for text in text_dataset
        ]
        text_similarities.sort(key=lambda x: x["similarity"])  

        music_pos_samples.append([anchor_index] + [x["index"] for x in music_similarities[:n]])
        music_neg_samples.append([anchor_index] + [x["index"] for x in music_similarities[-n:]])
        text_pos_samples.append([anchor_index] + [x["index"] for x in text_similarities[:n]])
        text_neg_samples.append([anchor_index] + [x["index"] for x in text_similarities[-n:]])

    return anchors, text_pos_samples, text_neg_samples, music_pos_samples, music_neg_samples

anchors, text_pos_samples, text_neg_samples, music_pos_samples, music_neg_samples = build_samples(
    text_dataset, music_dataset, n, num_anchors
)

train_size = int(len(text_pos_samples) * train_ratio)

train_text_pos_samples = text_pos_samples[:train_size]
test_text_pos_samples = text_pos_samples[train_size:]

train_text_neg_samples = text_neg_samples[:train_size]
test_text_neg_samples = text_neg_samples[train_size:]

train_music_pos_samples = music_pos_samples[:train_size]
test_music_pos_samples = music_pos_samples[train_size:]

train_music_neg_samples = music_neg_samples[:train_size]
test_music_neg_samples = music_neg_samples[train_size:]

save_samples_to_folder(train_text_pos_samples, "train_text_pos_samples.txt")
save_samples_to_folder(test_text_pos_samples, "test_text_pos_samples.txt")
save_samples_to_folder(train_text_neg_samples, "train_text_neg_samples.txt")
save_samples_to_folder(test_text_neg_samples, "test_text_neg_samples.txt")
save_samples_to_folder(train_music_pos_samples, "train_music_pos_samples.txt")
save_samples_to_folder(test_music_pos_samples, "test_music_pos_samples.txt")
save_samples_to_folder(train_music_neg_samples, "train_music_neg_samples.txt")
save_samples_to_folder(test_music_neg_samples, "test_music_neg_samples.txt")

print("Samples saved to", save_path)

Samples saved to D:/PolyU/URIS/Part2_projects/WEMOM_V1/Text2Music/Samples/


In [5]:
features1 = "D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/music_z.npy"
features2 = "D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/text_z.npy"


music = np.load(features1)
print(f"music shape:{music.shape}")  
text = np.load(features2)
print(f"text shape:{text.shape}")


music shape:(985, 256)
text shape:(6832, 128)
